<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i combine Fridata datset with SSI dataset & GPC dataset.**
*   *Frist combine FRIDA dataset with intrested columns*
*   *Second combine GPC dataset with CONCITO dataset with the help of Brickcode/GPC_level4_code*
*   *Then try to extract SSI dataset only the name and match with FRIDA dataset*
*   *Then introduce combined CONCITO_GOPC dataset to the Combined FRIODA_SSI dataset*

In [1]:
# Necessary libaries
import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore")

# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

***Import different dataset in FRIDA***

In [2]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Import FRIDA Group data with Group code
path_frida_data_groups="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Import FRIDA AdditionalLink data
path_frida_aditional_link="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaAdditionalLinkData.xlsx"

***Frida Nutrient dataset***

This Nutrient dataset is the **`*base dataset*`** to have snowballing approach and merge other dataset on the top of it.

In [225]:
# Lets open with little modificaton so easy to read and understand
frida_nutrient= pd.read_excel(path_frida_data, sheet_name=1, index_col=None, header=None, skiprows=2).round(decimals = 3).iloc[:, :26]

# Lets define and Assign columns name that defined just above
column_names= ['FoodID', 'FødevareGruppe', 'FødevareNavn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']
frida_nutrient.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
frida_nutrient.head(3).append(frida_nutrient.tail(3))

,FoodID,FødevareGruppe,FødevareNavn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
1183,1728,Fedtrig frugt og frø,Chiafrø,iv,1837,447,iv,3.425,21.4,21.4,...,0.887,iv,4.4,93.9,6.1,0,iv,0,0,0
1184,1729,Kornsorter,Perlespelt,iv,1386,329,iv,1.94,12.1,12.1,...,0.598,iv,1.5,88.8,11.2,0,iv,0,0,0
1185,1730,Kornsorter,Perlebyg,iv,1413,335,iv,1.455,9.1,9.1,...,0.548,iv,1.3,89.2,10.8,0,iv,0,0,0


In [226]:
frida_nutrient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   FoodID                       1186 non-null   int64 
 1   FødevareGruppe               1186 non-null   object
 2   FødevareNavn                 1186 non-null   object
 3   Svind_%                      1186 non-null   object
 4   Energy_kj                    1186 non-null   object
 5   Energy_kcal                  1186 non-null   object
 6   Nitrogen_til_protein_faktor  1186 non-null   object
 7   Nitrogen_total_g             1186 non-null   object
 8   Protein_videnskabelign_g     1186 non-null   object
 9   Protein_deklaration_g        1186 non-null   object
 10  Kulhydrat_difference_g       1186 non-null   object
 11  Kulhydrat_tilgængelig_g      1186 non-null   object
 12  Kulhydrat_deklaration_g      1186 non-null   object
 13  Tilsat_sukker_g              1186

***Frida group dataset***
*   In this dataset, we are interested to extract **`ParantFoodGroupId`** column
*   This dataset can only merge/Join with **`aditional_link dataset`** with column Common **`FoodGroupID`**

In [236]:
# Lets open data as pandas dataframe with adjusted column name
df_frida_groups= pd.read_excel(path_frida_data_groups, sheet_name=3, index_col=None, header=0, skiprows=0)[["FoodGroupID",	"GroupNameDK",	"GroupNameUK", "ParantFoodGroupId"]]
# lets have a look row with  "Banan, rå" to have understanding of the rows
df_frida_groups[df_frida_groups.ParantFoodGroupId == 47]

,FoodGroupID,GroupNameDK,GroupNameUK,ParantFoodGroupId
39,48,Anden frugt,Other fruit,47
40,49,Kernefrugt,Pome fruit,47
41,50,Stenfrugt,Stone fruit,47
42,51,Bærfrugt,Soft fruit,47
43,52,Sydfrugt,Tropical or subtropical fruit,47
44,53,Fedtrig frugt og frø,High-fat fruit and seeds,47
45,54,Frugtsaft og juice,Fruit juices,47
46,55,Frugtprodukter,Fruit products,47


In [237]:
df_frida_groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   FoodGroupID        148 non-null    int64 
 1   GroupNameDK        146 non-null    object
 2   GroupNameUK        146 non-null    object
 3   ParantFoodGroupId  148 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 4.8+ KB


***Frida Aditional_link dataset***
*   This dataset is useful for mainly two columns **`(TaxonomicName	FoodEx2Code)`**
*   This dataset can first merge with ***Group dataset*** with column **`FoodGroupID`**then after with ***Nutrient dataset***

In [238]:
# Lets open with little modificaton so easy to read and understand
df_frida_aditional_link= pd.read_excel(path_frida_aditional_link, sheet_name=3, index_col=None, header=0, skiprows=0)

# lets see first 3 and last 3 rows od the dataset
df_frida_aditional_link.head(3).append(df_frida_aditional_link.tail(3))

,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1248,"Oksekød, filet, marmoreret, rå","Beef, entrecote/rib eye, raw",1795,Bos taurus,NaN,58,Oksekød,Beef


In [239]:
df_frida_aditional_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   FødevareNavn    1249 non-null   object
 1   FoodName        1249 non-null   object
 2   FoodID          1249 non-null   int64 
 3   TaxonomicName   620 non-null    object
 4   FoodEx2Code     911 non-null    object
 5   FoodGroupID     1249 non-null   int64 
 6   FødevareGruppe  1249 non-null   object
 7   FoodGroup       1249 non-null   object
dtypes: int64(2), object(6)
memory usage: 78.2+ KB


**Combine `food_Group` and `aditional_link` dataset**

In [240]:
# also combine frida group dataset with aditional_link dataset
combinedf_aditionallink_groups= df_frida_aditional_link.merge(df_frida_groups[['FoodGroupID', 'ParantFoodGroupId']], on= 'FoodGroupID', how= 'left')
combinedf_aditionallink_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249 entries, 0 to 1248
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   FødevareNavn       1249 non-null   object 
 1   FoodName           1249 non-null   object 
 2   FoodID             1249 non-null   int64  
 3   TaxonomicName      620 non-null    object 
 4   FoodEx2Code        911 non-null    object 
 5   FoodGroupID        1249 non-null   int64  
 6   FødevareGruppe     1249 non-null   object 
 7   FoodGroup          1249 non-null   object 
 8   ParantFoodGroupId  1104 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 97.6+ KB


***Compare two dataset to understand merging options***

In [241]:
# Compare Nutrient vs Frida-aditional_link dataset and find the rows that are not exist in eachother 

# with "FødevareNavn" column
print("Compare with 'FødevareNavn' column")
print("Only exist on Frida Nutrient dataset:", frida_nutrient[~frida_nutrient['FødevareNavn'].isin(combinedf_aditionallink_groups["FødevareNavn"])].shape)
print("Only exist on Frida-aditional_link:", combinedf_aditionallink_groups[~combinedf_aditionallink_groups['FødevareNavn'].isin(frida_nutrient["FødevareNavn"])].shape)
print()

# with "FødevareGruppe" column
print("Compare with 'FødevareGruppe' column")
print("Only exist in Nitrient dataset:", frida_nutrient[~frida_nutrient['FødevareGruppe'].isin(combinedf_aditionallink_groups["FødevareGruppe"])].shape)
print("Only exist in Frida-aditional_link:", combinedf_aditionallink_groups[~combinedf_aditionallink_groups['FødevareGruppe'].isin(frida_nutrient["FødevareGruppe"])].shape)
print()

# with "FoodID "column
print("Compare with 'FoodID' column")
print("Only exist in Nitrient dataset:", frida_nutrient[~frida_nutrient['FoodID'].isin(combinedf_aditionallink_groups["FoodID"])].shape)
print("Only exist in Frida-aditional_link:", combinedf_aditionallink_groups[~combinedf_aditionallink_groups['FoodID'].isin(frida_nutrient["FoodID"])].shape)

Compare with 'FødevareNavn' column
Only exist on Frida Nutrient dataset: (45, 26)
Only exist on Frida-aditional_link: (108, 9)

Compare with 'FødevareGruppe' column
Only exist in Nitrient dataset: (442, 26)
Only exist in Frida-aditional_link: (477, 9)

Compare with 'FoodID' column
Only exist in Nitrient dataset: (4, 26)
Only exist in Frida-aditional_link: (67, 9)


***Sumarry of two dataset from FRIDA:`Frida_Nutrient` and `combine_aditionallink_groups_dataset`****** 

When comparing columns in two dataset of ***Frida_Nutrient*** and ***combine_aditionallink_groups_dataset***, the matching column ***FoodID*** gives the maximun matching rows so i used FoodID as a merging column
*   ***4 rows*** in `data_frida['FoodID']` are not in `data_frida_aditional_link["FoodID"]`. 
*   on the otherhand, ***67 rows*** in `combine_aditionallink_groups_dataset["FoodID"]` are not in `data_frida['FoodID']` 
*   So, i consider ***data_frida as a base dataset*** and only merge the rows that exist in data_frida. Which can be done with ***left join***.


In [242]:
# Frida Additionallink dataset & Nutrient Dataset
finaldf_fridanutrient_aditionallink_groups= frida_nutrient.merge(combinedf_aditionallink_groups, on= 'FoodID', how= 'left')
finaldf_fridanutrient_aditionallink_groups

,FoodID,FødevareGruppe_x,FødevareNavn_x,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,...,D_vitamin_µg,D3_vitamin_µg,FødevareNavn_y,FoodName,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe_y,FoodGroup,ParantFoodGroupId
0,1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,...,0,0,"Jordbær, rå","Strawberry, raw",Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit,47.0
1,2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,...,0,0,"Æble, uspec., råt","Apple, raw, all varieties",Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49.0,Kernefrugt,Pome fruit,47.0
2,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,0,0,"Banan, rå","Banana, raw",Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit,47.0
3,4,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,...,0,0,"Kartoffel, uspec., rå","Potato, raw",Solanum tuberosum L.,A00ZT#F02.A067V,39.0,Rod- og knoldgrøntsager,Root and tuber vegetables,37.0
4,5,Bærfrugt,"Hindbær, rå",0,215,51,6.25,0.219,1.4,1.4,...,0,0,"Hindbær, rå","Raspberry, raw",Rubus idaeus L.,A01EP#F02.A067D,51.0,Bærfrugt,Soft fruit,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,1726,Kornsorter,"Quinoa, sort, rå",iv,1461,347,iv,2.185,13.7,13.7,...,0,0,"Quinoa, sort, rå","Quinoa, black, raw",Chenopodium quinoa,NaN,28.0,Korn og gryn,Grain and groats,26.0
1182,1727,Mel,"Rugmel, uspec.",iv,1358,322,iv,1.385,8.7,8.7,...,0,0,"Rugmel, uspec.","Rye flour, unspecified",Secale cereale L.,A003J,30.0,Mel og klid,Flour and bran,26.0
1183,1728,Fedtrig frugt og frø,Chiafrø,iv,1837,447,iv,3.425,21.4,21.4,...,0,0,Chiafrø,Chia seeds,Salvia hispanica,A16FZ,177.0,Fedtrige frø,High-fat seeds,NaN
1184,1729,Kornsorter,Perlespelt,iv,1386,329,iv,1.94,12.1,12.1,...,0,0,Perlespelt,Pearled spelt,Triticum spelta,A001R,28.0,Korn og gryn,Grain and groats,26.0


In [243]:
finaldf_fridanutrient_aditionallink_groups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1186 entries, 0 to 1185
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   FoodID                       1186 non-null   int64  
 1   FødevareGruppe_x             1186 non-null   object 
 2   FødevareNavn_x               1186 non-null   object 
 3   Svind_%                      1186 non-null   object 
 4   Energy_kj                    1186 non-null   object 
 5   Energy_kcal                  1186 non-null   object 
 6   Nitrogen_til_protein_faktor  1186 non-null   object 
 7   Nitrogen_total_g             1186 non-null   object 
 8   Protein_videnskabelign_g     1186 non-null   object 
 9   Protein_deklaration_g        1186 non-null   object 
 10  Kulhydrat_difference_g       1186 non-null   object 
 11  Kulhydrat_tilgængelig_g      1186 non-null   object 
 12  Kulhydrat_deklaration_g      1186 non-null   object 
 13  Tilsat_sukker_g   

#CONCITO dataset and GPC dataset

***Import dataset***

In [12]:
# Climate dataset
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

#TradeSync GPC dataset
path_tradesync_excel= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/GS1_GPC_Dataset.xlsx" 

***CONCITO climate dataset***

In [13]:
# Loading CONCITO climate dataset into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq_perkg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]
cols_to_use= ['Product_dk',	"Product_en", 'Category_dk', "Category_en", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq_perkg"]

# Original columns name are replaced with ralavant names (col_names) and open only selected columns
data_climate_selected= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).loc[:, cols_to_use]
data_climate_selected.head(5)

,Product_dk,Product_en,Category_dk,Category_en,GPC_Level4_dk,Fat_g,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq_perkg
0,"Peberfrugt, rød, rå","Pepper, sweet, red, raw",Grøntsager,Vegetables,10006191,0.1,5.2,0.9,133.0,0.251618,0.033544,0.000000e+00,0.140624,0.582215,0.009921,1.017923
1,"Tomat, uspec., rå","Tomato, ripe, raw, origin unknown",Grøntsager,Vegetables,10006165,0.1,3.2,0.8,87.0,0.066407,0.008081,0.000000e+00,0.140624,0.476213,0.009921,0.701246
2,"Squash, rå","Squash, raw",Grøntsager,Vegetables,10006355,0.2,2.4,1.4,81.0,0.141884,0.040487,0.000000e+00,0.064813,0.577676,0.009921,0.834780
3,"Aubergine, rå","Aubergine, raw",Grøntsager,Vegetables,10006128,0.1,3.1,0.9,83.0,0.158010,0.019844,0.000000e+00,0.140624,0.646277,0.009921,0.974676
4,"Tomat, soltørret","Tomato, dried",Grøntsager,Vegetables,10000272,3.0,43.5,14.1,1187.0,0.905904,0.091830,-5.551115e-17,0.260522,0.677568,0.009921,1.945744


In [14]:
data_climate_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product_dk          500 non-null    object 
 1   Product_en          500 non-null    object 
 2   Category_dk         500 non-null    object 
 3   Category_en         500 non-null    object 
 4   GPC_Level4_dk       500 non-null    object 
 5   Fat_g               500 non-null    float64
 6   Carb_g              500 non-null    object 
 7   Protein_g           500 non-null    object 
 8   Energy_KJ           500 non-null    float64
 9   Agriculture         500 non-null    float64
 10  iLUC                500 non-null    float64
 11  Processing          500 non-null    float64
 12  Packaging           500 non-null    float64
 13  Transport           500 non-null    float64
 14  Retail              500 non-null    float64
 15  Total_CO2_eq_perkg  500 non-null    float64
dtypes: float

***GS1 TradeSync GPC dataset***

In [15]:
# Selective columns names
columns = ['SegmentCode', 'SegmentTitle', 'FamilyCode', 'FamilyTitle', 'ClassCode', 'ClassTitle', 'BrickCode', 'BrickTitle', 'AttributeCode', 'AttributeTitle', 'AttributeValueCode', 'AttributeValueTitle']

# Lets open data as they are
data_tradesync_excel= pd.read_excel(path_tradesync_excel, sheet_name=0, index_col=None, header=0, skiprows=0)
data_tradesync_excel.head(2).append(data_tradesync_excel.tail(2))

# Open the dataset with selective columns names
data_tradesync_selected_columns = data_tradesync_excel[columns]
data_tradesync_selected_columns.head(2).append(data_tradesync_selected_columns.tail(2))


,SegmentCode,SegmentTitle,FamilyCode,FamilyTitle,ClassCode,ClassTitle,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle
0,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001686,Airbrushes (Powered),NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,30008542.0,AIRBRUSH CONTROL VALVE
183206,77000000,Vehicle,77030000,Cars and Motorcycles,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,30002518.0,UNIDENTIFIED
183207,77000000,Vehicle,77050000,Specialty Vehicles,77050100,Specialty Vehicles,10008050,Specialty Vehicles,NaN,NaN,NaN,NaN


In [16]:
#data_tradesync_excel.info()
data_tradesync_selected_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183208 entries, 0 to 183207
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SegmentCode          183208 non-null  int64  
 1   SegmentTitle         183208 non-null  object 
 2   FamilyCode           183208 non-null  int64  
 3   FamilyTitle          183208 non-null  object 
 4   ClassCode            183208 non-null  int64  
 5   ClassTitle           183208 non-null  object 
 6   BrickCode            183208 non-null  int64  
 7   BrickTitle           183208 non-null  object 
 8   AttributeCode        181646 non-null  float64
 9   AttributeTitle       181646 non-null  object 
 10  AttributeValueCode   181646 non-null  float64
 11  AttributeValueTitle  181646 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 16.8+ MB


In [17]:
#data_tradesync_selected_columns[data_tradesync_selected_columns.BrickCode== 10006191]	
#data_climate_selected[data_climate_selected.GPC_Level4_dk== 10006191]	

***Now, combine CONCITO_Climate & GS1_GPC_Tradesync datasets***

In [18]:
# CONCITO dataset with GPC dataset
combine_gpc_climate_dataset= data_tradesync_selected_columns.merge(data_climate_selected, left_on= 'BrickCode', right_on= 'GPC_Level4_dk', how= 'left')
combine_gpc_climate_dataset.head(2).append(combine_gpc_climate_dataset.tail(2))
#combine_gpc_climate_dataset.info()


,SegmentCode,SegmentTitle,FamilyCode,FamilyTitle,ClassCode,ClassTitle,BrickCode,BrickTitle,AttributeCode,AttributeTitle,...,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq_perkg
0,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001686,Airbrushes (Powered),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227356,77000000,Vehicle,77030000,Cars and Motorcycles,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227357,77000000,Vehicle,77050000,Specialty Vehicles,77050100,Specialty Vehicles,10008050,Specialty Vehicles,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
combine_gpc_climate_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227358 entries, 0 to 227357
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SegmentCode          227358 non-null  int64  
 1   SegmentTitle         227358 non-null  object 
 2   FamilyCode           227358 non-null  int64  
 3   FamilyTitle          227358 non-null  object 
 4   ClassCode            227358 non-null  int64  
 5   ClassTitle           227358 non-null  object 
 6   BrickCode            227358 non-null  object 
 7   BrickTitle           227358 non-null  object 
 8   AttributeCode        225795 non-null  float64
 9   AttributeTitle       225795 non-null  object 
 10  AttributeValueCode   225795 non-null  float64
 11  AttributeValueTitle  225795 non-null  object 
 12  Product_dk           83629 non-null   object 
 13  Product_en           83629 non-null   object 
 14  Category_dk          83629 non-null   object 
 15  Category_en      

***Exploring connecting column and feacture for FRIDA and GPC_Concito dataset***

In [20]:
combine_gpc_climate_dataset[combine_gpc_climate_dataset.Product_dk == "Jordbær, rå"].iloc[:, [6,7,8,9,10,11,12,13,27]]

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,Total_CO2_eq_perkg
66614,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014610.0,AFGHANISTAN,"Jordbær, rå","Strawberry, raw",0.561566
66615,10005921,Strawberries,20000743.0,Country/Zone of Origin,30016053.0,ÅLAND ISLANDS,"Jordbær, rå","Strawberry, raw",0.561566
66616,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014612.0,ALBANIA,"Jordbær, rå","Strawberry, raw",0.561566
66617,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014613.0,ALGERIA,"Jordbær, rå","Strawberry, raw",0.561566
66618,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014614.0,AMERICAN SAMOA,"Jordbær, rå","Strawberry, raw",0.561566
...,...,...,...,...,...,...,...,...,...
66896,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014956.0,"""EXTRA"" CLASS","Jordbær, rå","Strawberry, raw",0.561566
66897,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014608.0,CLASS I,"Jordbær, rå","Strawberry, raw",0.561566
66898,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014609.0,CLASS II,"Jordbær, rå","Strawberry, raw",0.561566
66899,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30002515.0,UNCLASSIFIED,"Jordbær, rå","Strawberry, raw",0.561566


In [21]:
combine_fridanutrient_additionallink_dataset #[combine_fridanutrient_additionallink_dataset.FødevareNavn == "Jordbær, rå"].info()

,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup,Svind_%,Energy_kj,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit,4,162,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit,10,233,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit,41,396,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
3,"Kartoffel, uspec., rå","Potato, raw",4,Solanum tuberosum L.,A00ZT#F02.A067V,39,Rod- og knoldgrøntsager,Root and tuber vegetables,25,326,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0
4,"Hindbær, rå","Raspberry, raw",5,Rubus idaeus L.,A01EP#F02.A067D,51,Bærfrugt,Soft fruit,0,215,...,0.8,0,0.5,13.8,86.2,3.5,0,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,"Pølse, med ærteprotein","Sausage, with pea protein",1791,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1245,"Pålæg i skiver, med æggehvide","Cold cuts, with eggwhite",1792,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Combine Frida (Combined) and gs1GPC-climate dataset
*   ***Product_dk* from combined GS1_GPC dataset. This dataset have multiple datapoint for selected string from FRIDA**
*   ***FødevareNavn* from frida dataset- This dataset have one 1 datapoint for selected string**


So it need to be merged FRIDA dataset to GPC-Concito dataset. 
After inspecting dataset:
*   Single "BrickCode" contain multiple rows with multiple AttributeCode and AttributeValue
*   It will make sense if we pass single food item with BrickCode and AttributeValueCode. because there are multiple BrickCode but single AttributeValueCode

In [22]:
# explore seleted columns that seems relavant
combine_fida_gpc_concito_dataset= combine_gpc_climate_dataset.merge(combine_fridanutrient_additionallink_dataset, left_on= 'Product_dk', right_on= 'FødevareNavn', how= 'left')
combine_fida_gpc_concito_dataset.head(2).append(combine_fida_gpc_concito_dataset.tail(2)).iloc[:, [6,7,8,9,10,11,12,13,27,30,31,32,33,34,35]]

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,Total_CO2_eq_perkg,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
0,10001686,Airbrushes (Powered),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,30008542.0,AIRBRUSH CONTROL VALVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227356,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,30002518.0,UNIDENTIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227357,10008050,Specialty Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
combine_fida_gpc_concito_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227358 entries, 0 to 227357
Data columns (total 59 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SegmentCode                  227358 non-null  int64  
 1   SegmentTitle                 227358 non-null  object 
 2   FamilyCode                   227358 non-null  int64  
 3   FamilyTitle                  227358 non-null  object 
 4   ClassCode                    227358 non-null  int64  
 5   ClassTitle                   227358 non-null  object 
 6   BrickCode                    227358 non-null  object 
 7   BrickTitle                   227358 non-null  object 
 8   AttributeCode                225795 non-null  float64
 9   AttributeTitle               225795 non-null  object 
 10  AttributeValueCode           225795 non-null  float64
 11  AttributeValueTitle          225795 non-null  object 
 12  Product_dk                   83629 non-null   object 
 13 

In [24]:
# lets see the row with "Jordbær, rå"
a=combine_fida_gpc_concito_dataset[combine_fida_gpc_concito_dataset.BrickCode == 10005897].iloc[:, [6,7,8,9,10,11,12,13,27,30,31,32,33,34,35]]
a

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,Total_CO2_eq_perkg,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
60078,10005897,Bananas,20000743.0,Country/Zone of Origin,30014610.0,AFGHANISTAN,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60079,10005897,Bananas,20000743.0,Country/Zone of Origin,30016053.0,ÅLAND ISLANDS,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60080,10005897,Bananas,20000743.0,Country/Zone of Origin,30014612.0,ALBANIA,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60081,10005897,Bananas,20000743.0,Country/Zone of Origin,30014613.0,ALGERIA,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60082,10005897,Bananas,20000743.0,Country/Zone of Origin,30014614.0,AMERICAN SAMOA,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60358,10005897,Bananas,20002739.0,Growing Method,30014684.0,TRANSITIONAL,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60359,10005897,Bananas,20002739.0,Growing Method,30002518.0,UNIDENTIFIED,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60360,10005897,Bananas,20002793.0,If Ripened,30002960.0,NO,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit
60361,10005897,Bananas,20002793.0,If Ripened,30002518.0,UNIDENTIFIED,"Banan, rå","Banana, raw",0.982348,3.0,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52.0,Sydfrugt,Tropical or subtropical fruit


In [25]:
from google.colab import files
#combine_fida_gpc_concito_dataset.to_excel('combine_fida_gpc_concito_dataset.xlsx', encoding = 'utf-8-sig') 
#files.download('combine_fida_gpc_concito_dataset.xlsx')

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [26]:
# Import data
path_ssi_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_ssi= pd.read_csv(path_ssi_data, sep= ";", index_col=False, encoding='latin-1')

# itemnumber culumn is string although it is integer, lets turn it to integer
#But there are NULL value so can not converted to integer type
#data_ssi["itemnumber"]= data_ssi["itemnumber"].astype(int)

# lets see frist 3 and last 3 rows od the dataset
data_ssi.head(3).append(data_ssi.tail(5))

# only some of the rows of different kinds of the banana
b=data_ssi.iloc[[11, 12, 14, 18], :] 
b

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


In [27]:
#data_ssi.at[14,'brick_code']=0
data_ssi.iloc[[11, 12, 14, 18], :] 

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


In [28]:
data_ssi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112766 entries, 0 to 112765
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   itemname           112701 non-null  object
 1   itemnumber         112613 non-null  object
 2   category           71516 non-null   object
 3   percent_purchases  112728 non-null  object
dtypes: object(4)
memory usage: 3.4+ MB


In [29]:
# lets explore the SSI dataset with GTIN code from GS1

#iterate over gtin and Brick-code dataset
data_ssi["brick_code"] =""
for row, col in gtin_brickcode.iterrows():
  gtin_list= col.gtin #["5712873336944", '	5711044628659', '5707402006641', '406', '5711044628659', '5707402008119', '5707402008171', '0000057014388']
  gpc_code= col.gpcCategoryCode[0]
  #print("gtin:", gtin_list)
  #print("brik:", gpc_code)
  #print("-"*40)

  # now lets take a row from SSI dataset
  for  row, col in data_ssi.iloc[:,:].iterrows():    
    # check if itemnumber from SSI rows is exist in the column of GTIN code list
    if col.itemnumber in gtin_list:
      data_ssi.at[row,'brick_code']=gpc_code
      print(data_ssi.brick_code.unique())
    else:
      pass
  #print("*"* 40)
  

NameError: ignored

In [ ]:
data_ssi.iloc[[113,118,596], :]

In [ ]:
data_ssi.to_excel('data_ssi.xlsx', encoding = 'utf-8-sig') 
files.download('data_ssi.xlsx')

#Combine SSI data with Combined Frida-GS1GPC-climate dataset
*   **"itemname" from SSI dataset and "FødevareNavn" from Combined Frida-GPC-CONCITO dataset**
*    **Merging or Joining whole dataset do not making sense here, so i pick one row from SSI dataset and match with other dataset based on best match with Record Link library**

In [ ]:
!pip install recordlinkage

In [ ]:
# comparing the two dataframe with Record Link
# Import record linkage
import recordlinkage

#Create indexing object
#indexer= recordlinkage.Index()
#indexer.full()
#indexer.sortedneighbourhood(left_on= "itemname", right_on= "Product_dk")

#Generate pairs blocked on column
#indexer.block(left_on="itemnumber", right_on= "gtin")
candidates= indexer.index(b, a)
compare= recordlinkage.Compare()

print(len(candidates))

# Find exact matches for pairs of columns
compare.string("itemname", "Product_dk", threshold= 0.85, label= "Name")
potential_match= compare.compute(candidates, data_ssi, combine_fida_gpc_concito_dataset)
potential_match.Name.unique()

# TradeSync GTIN Link data from GS1 

In [ ]:
import glob
import xml.etree.ElementTree as ET

#Import gs1 xmll files
path_tradesync_data= "/content/drive/MyDrive/Data_gs1_xml/"
xmls = glob.glob(path_tradesync_data + '*.xml')
print("Toal amount of xml files: ", len(xmls))

In [ ]:
# Funccction to extract data from xml files
def gs1_xml_reader(file):
  """This function will take xml file and return GTIN and Brick code as key:value pair including other relavant feactures"""
  tree = ET.parse(file)
  root = tree.getroot()
  keys_of_items_to_find= ("gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', "additionalTradeItemIdentification", "uniformResourceIdentifier", "ingredientStatement")
  
  #Lets start the dictionary to collecct items 
  dit= dict([(key,[]) for key in keys_of_items_to_find])

  # Loop over the tags that make sense (from list above)
  for list_item in keys_of_items_to_find:
    # Again  looping to get the text value
    for item in root.iter(list_item):
      if item.text not in dit[list_item]:
        if list_item == "gtin":
          dit[list_item].append(item.text[1:])
        else:
          dit[list_item].append(item.text)
      #else:
        #dit[list_item].append(0)
  # The data are Dictionary of lists so lets print them to see
  return dit

gs1_xml_reader(xmls[0])

***The above data are in a form of dictionary. Dictionary of list.***

In [ ]:
print(type(gs1_xml_reader(xmls[0])))
print(type(gs1_xml_reader(xmls[0])['gtin']))


In [ ]:
# Lets loop over all the files and extract the GTIN code and associated Brick-codes
keys_of_items_to_find= ("gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', "additionalTradeItemIdentification", "uniformResourceIdentifier", "ingredientStatement")
dit= dict([(key,[0]) for key in keys_of_items_to_find])
df_with_gtin_list = pd.DataFrame()
for xml in xmls[:]:
  df_with_gtin_list= df_with_gtin_list.append(gs1_xml_reader(xml), ignore_index = True)
  
df_with_gtin_list

In [ ]:
#df_with_gtin_list.loc[1059, "gpcCategoryCode"]

gtin_brickcode= df_with_gtin_list[["gtin",	"gpcCategoryCode"]]
gtin_brickcode


In [ ]:
df_with_gtin_list[df_with_gtin_list.brandName == '[Naturli]']#.gpcCategoryCode.values

In [ ]:
combine_fida_gpc_concito_dataset[combine_fida_gpc_concito_dataset.FødevareNavn == "Jordbær, rå"].iloc[:, [6,7,8,9,10,11,12,13,16,27,28,29,30,31,32,33,34,35]]


In [ ]:
combine_fida_gpc_concito_dataset.shape

In [ ]:
combine_fida_gpc_concito_dataset[combine_fida_gpc_concito_dataset.FødevareNavn == "Jordbær, rå"].iloc[:, [6,7,12,13,27,30,31,32,33,34,35]].drop_duplicates()

In [ ]:
from google.colab import files
df_with_gtin_list= df_with_gtin_list[["gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', 'additionalTradeItemIdentification', "uniformResourceIdentifier","ingredientStatement"]].iloc[:,:]
df_with_gtin_list.to_excel('df_with_gtin_list.xlsx', encoding = 'utf-8-sig') 
#files.download('df_with_gtin_list.xlsx')

#Combine SSI data and GTIN:BRICK-CODE dataset
*   ***FødevareNavn* from frida dataset- This dataset have one 1 datapoint for selected string**
*   ***BrickTitle* from GS1_GPC dataset. This sdataset have multiple datapoint for selected string from FRIDA.FødevareNavn**

So it need to be merged FRIDA dataset to GPC-Concito dataset. 

In [ ]:
gs1_gtin= [05000387905498, 05000387905634]
type(gs1_gtin)

#Lets create the pipeline to extract data from the combination of ISS dataset, Frida dataset and GS1 GPC-dataset

In [ ]:
!pip3 install eansearch
from eansearch import EANSearch

ean_list= ['05701018044677', '05000387905634', 25109, 5712873336944, 5712580050171, 20008932, 20014018, 20023393, 5712580000114, "05766632526800"]
ean = ean_list[0]
apiToken = "a09851e102099d04448394e0b838bc345c8fd98a"


"""eansearch = EANSearch(apiToken)
eanList = eansearch.productSearch('Bananas')
for product in eanList:
	print(product["ean"], " is ", product["name"].encode("utf-8"))"""
 
lookup = EANSearch(apiToken)
name = lookup.barcodeLookup(ean)
print(ean, " is ", name)